# Наивный Байес

Импортирую нужные библиотеки и загружаю датасет.

In [56]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

messages = pandas.read_csv('SMSSpamCollection', sep='\t', names=["label", "message"])

In [10]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [47]:
print(messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


Как показывает группировка данных по классам, датасет не сбалансированный, так как спама сильно больше, чем не спама. Если всем новым наблюдениям присваивать класс ham, то точность будет достаточно высокой, но с точки зрения задачи классификации спами это плохое решение, так как такой классификатор вообще не будет отсеивать спам. 

**Балансирую данные**

In [48]:
spam = messages[messages['label'] == 'spam']

In [49]:
spam.head()

,label,message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [62]:
ham = messages[messages['label'] == 'ham']

In [28]:
print(len(ham))

4825


In [61]:
less_ham = ham.sample(n=len(spam))
print(len(less_ham))

747


In [80]:
balanced = pandas.concat([less_ham, spam])
print(len(balanced))

1494


**Токенизирую**

In [81]:
def tokenize_leave_punct(text): return word_tokenize(text.lower())
def tokenize_remove_punct(text):
    text = word_tokenize(text.lower())
    punct = '!.,:;"?'
    for p in punct:
        if p in text:
            text.remove(p)
    return text

In [88]:
with_punct, without_punct = copy.deepcopy(balanced), copy.deepcopy(balanced)
with_punct.message = with_punct.message.apply(tokenize_leave_punct)
with_punct.head()

,label,message
4674,ham,"[i, forgot, 2, ask, ü, all, smth.., there, 's,..."
220,ham,"[yeah, you, should, ., i, think, you, can, use..."
1664,ham,"[s, but, mostly, not, like, that, .]"
332,ham,"[maybe, i, could, get, book, out, tomo, then, ..."
4343,ham,"[ha, !, i, would, n't, say, that, i, just, did..."


In [89]:
without_punct.message = without_punct.message.apply(tokenize_remove_punct)
without_punct.head()

,label,message
4674,ham,"[i, forgot, 2, ask, ü, all, smth.., there, 's,..."
220,ham,"[yeah, you, should, i, think, you, can, use, y..."
1664,ham,"[s, but, mostly, not, like, that]"
332,ham,"[maybe, i, could, get, book, out, tomo, then, ..."
4343,ham,"[ha, i, would, n't, say, that, i, just, did, n..."


**Лемматизирую**